In [1]:
import os

outfile_directory = os.path.join('data', 'outfiles')
file_location = os.path.join('data','outfiles','*.out')
print(file_location)

data\outfiles\*.out


In [2]:
import glob
filenames = glob.glob(file_location)
print(filenames)

['data\\outfiles\\butanol.out', 'data\\outfiles\\decanol.out', 'data\\outfiles\\ethanol.out', 'data\\outfiles\\heptanol.out', 'data\\outfiles\\hexanol.out', 'data\\outfiles\\methanol.out', 'data\\outfiles\\nonanol.out', 'data\\outfiles\\octanol.out', 'data\\outfiles\\pentanol.out', 'data\\outfiles\\propanol.out']


In [3]:
for f in filenames:
    outfile = open(f,'r')
    data = outfile.readlines()
    outfile.close()
    
# ok, let's get the energies out of all our files

    for line in data:
        if 'Final Energy' in line:
            energy_line = line
            words = energy_line.split()
            energy = float(words[3])
            print(energy)

-232.1655798347283
-466.3836241400086
-154.09130176573018
-349.27397687072676
-310.2385332251633
-115.04800861868374
-427.3465180082815
-388.3110864554743
-271.20138119895074
-193.12836249728798


In [4]:
first_file = filenames[0]
print(first_file)

file_name = os.path.basename(first_file)
print(file_name)

split_filename = file_name.split('.')
molecule_name = split_filename[0]
print(molecule_name)

data\outfiles\butanol.out
butanol.out
butanol


In [5]:
# we open/create a file for writing (w+)
# use a+ to append
datafile = open('energies.txt','w+')
for f in filenames:
    # get molecule name
    file_name = os.path.basename(f)
    split_filename = file_name.split('.')
    molecule_name = split_filename[0]

    # read data
    outfile = open(f,'r')
    data = outfile.readlines()
    outfile.close()

    # loop thru
    for line in data:
        if 'Final Energy' in line:
            energy_line = line
            words = energy_line.split()
            energy = float(words[3])
            datafile.write(F'{molecule_name} \t {energy} \n')
            # F' is F string notation. \n is a newline character to keep text from smushing
datafile.close()

In [6]:
# 03_Prod.mdout Etot.txt exercise

datafile = open('Etot.txt','w+')
prod = os.path.join('data','*.mdout')
prod = glob.glob(prod)
outfile = open(prod[0],'r')
data = outfile.readlines()
outfile.close()

for line in data:
    if 'Etot   ' in line:
        Etot_line = line
        words = Etot_line.split()
        Etot = float(words[2])
        datafile.write(F'{Etot} \n')
datafile.close()

In [7]:
import numpy
# we are interested here in genfromtxt()
    # we take a file and turn it into a list of lists
    # mixed data types --> unicode dtype, csv --> delimiter = ','
# I don't need to import os again as I already have earlier

distance_file = os.path.join('data','distance_data_headers.csv')
distances = numpy.genfromtxt(fname = distance_file, delimiter = ',', dtype = 'unicode')
# printing distances will give us everything as a list of lists
headers = distances[0]
data = distances[1:]
data = data.astype(numpy.single)
# printing data in this form will give us a list of list of all the floats
# array_name[row, column] therefore data[0,1] is 8.9542
print(data[0,1])
small_data = data[0:10,0:3]
print(small_data)

thr4_atp = data[:,1]  # Every row, just the THR4_ATP column
avg_thr4_atp = numpy.mean(thr4_atp)
print(avg_thr4_atp)

8.9542
[[ 1.      8.9542  5.8024]
 [ 2.      8.6181  6.0942]
 [ 3.      9.0066  6.0637]
 [ 4.      9.2002  6.0227]
 [ 5.      9.1294  5.9365]
 [ 6.      9.0462  6.2553]
 [ 7.      8.8657  5.9186]
 [ 8.      9.3256  6.2351]
 [ 9.      9.4184  6.1993]
 [10.      9.06    6.0478]]
10.87695


In [8]:
num_columns = len(data[0])
for i in range(1, num_columns):
    column = data[:,i]
    avg_column = numpy.mean(column)
    print(F'{headers[i]}: {avg_column}')

THR4_ATP: 10.87695026397705
THR4_ASP: 7.342345237731934
TYR6_ATP: 11.209792137145996
TYR6_ASP: 10.993443489074707


In [9]:
# we want to read water.xyz and get atom distances
datafile = open('bond_lengths.txt','w+')
file_location = os.path.join('data', 'water.xyz')
output = numpy.genfromtxt(fname=file_location, skip_header=2,  dtype='unicode')
symb = output[:,0] # atom label
coords = (output[:,1:])
coords = coords.astype(numpy.single)
num_atoms = len(symb) # how many atoms are there
for num1 in range(0,num_atoms):
    for num2 in range(0,num_atoms):
        if num1 < num2: # prevents double counting (O to H1 vs H1 to O reduced down to 1)
            x_distance = coords[num1,0] - coords[num2,0]
            y_distance = coords[num1,1] - coords[num2,1]
            z_distance = coords[num1,2] - coords[num2,2]
            bond_length_12 = numpy.sqrt(x_distance**2+y_distance**2+z_distance**2)
            if bond_length_12 > 0 and bond_length_12 <= 1.5: # no self-atom counting and no counting non-bonds
                datafile.write(F'{symb[num1]} to {symb[num2]} : {bond_length_12:.3f}\n') # .3f = 3 sigfigs
datafile.close()